In [1]:
import requests
import pandas as pd
import datetime

# API Endpoint
API_URL = "https://api.exchangerate.host/timeseries"
START_DATE = "2015-01-01"
END_DATE = datetime.date.today().strftime("%Y-%m-%d")

# Fetch INR to USD exchange rates
params = {
    "start_date": START_DATE,
    "end_date": END_DATE,
    "base": "INR",
    "symbols": "USD"
}

response = requests.get(API_URL, params=params)
data = response.json()

# Convert to DataFrame
if "rates" in data:
    df = pd.DataFrame.from_dict(data["rates"], orient="index")
    df.index = pd.to_datetime(df.index)
    df.to_csv("inr_usd_exchange.csv")
    print("Data saved successfully!")
else:
    print("Failed to fetch data.")


Failed to fetch data.


In [4]:
import requests
import pandas as pd
import datetime

# API Endpoint
API_URL = "https://api.exchangerate.host/timeseries"
START_DATE = "2015-01-01"
END_DATE = datetime.date.today().strftime("%Y-%m-%d")

# Fetch INR to USD exchange rates
params = {
    "start_date": START_DATE,
    "end_date": END_DATE,
    "base": "INR",
    "symbols": "USD"
}

response = requests.get(API_URL, params=params)
data = response.json()

# Convert to DataFrame
# Check if the API call was successful before proceeding
if data.get("success"):  # Use get() to avoid potential KeyError if "success" is missing
    df = pd.DataFrame.from_dict(data["rates"], orient="index")
    df.index = pd.to_datetime(df.index)
    # Rename the column to 'USD'
    df = df.rename(columns={df.columns[0]: 'USD'}) # Assuming 'INR_to_USD' or similar is the first column
    df.to_csv("inr_usd_exchange.csv")
    print("Data saved successfully!")
else:
    print(f"Failed to fetch data: {data.get('error')}") # Print the error message for debugging

Failed to fetch data: {'code': 101, 'type': 'missing_access_key', 'info': 'You have not supplied an API Access Key. [Required format: access_key=YOUR_ACCESS_KEY]'}


In [6]:
# Fetch EUR to USD exchange rate
params["base"] = "EUR"
response = requests.get(API_URL, params=params)
data_eur = response.json()

# Check if the API call was successful before proceeding
if data_eur.get("success"):
    df_eur = pd.DataFrame.from_dict(data_eur["rates"], orient="index")
    df_eur.index = pd.to_datetime(df_eur.index)
else:
    print(f"Failed to fetch EUR to USD data: {data_eur.get('error')}")
    # Handle the error, e.g., exit or use default values

# ... (rest of the code)

Failed to fetch EUR to USD data: {'code': 101, 'type': 'missing_access_key', 'info': 'You have not supplied an API Access Key. [Required format: access_key=YOUR_ACCESS_KEY]'}


In [ ]:
def batch_analysis(df, batch_size):
    years = sorted(df.index.year.unique())
    for i in range(0, len(years), batch_size):
        batch_years = years[i:i+batch_size]
        batch_data = df[df.index.year.isin(batch_years)]

        print(f"Analyzing Batch: {batch_years}")
        print(batch_data.describe())  # Summary statistics

        # Plot batch trend
        plt.figure(figsize=(8, 4))
        plt.plot(batch_data.index, batch_data["USD"], label=f"INR to USD {batch_years}", alpha=0.6)
        plt.xlabel("Year")
        plt.ylabel("Exchange Rate")
        plt.title(f"INR to USD Trend ({batch_years})")
        plt.legend()
        plt.show()

# Analyze in batches of 2 years
batch_analysis(df, batch_size=2)


In [ ]:
from multiprocessing import Pool

# Define a function to process each year's data
def analyze_year(year):
    subset = df[df.index.year == year]
    trend = subset["USD"].mean()
    return (year, trend)

years = sorted(df.index.year.unique())

# Use multiprocessing to process data in parallel
with Pool() as pool:
    results = pool.map(analyze_year, years)

# Convert results to a DataFrame
df_trends = pd.DataFrame(results, columns=["Year", "Average Exchange Rate"])


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_trends["Year"], df_trends["Average Exchange Rate"], marker="o", linestyle="-", color="blue")
plt.xlabel("Year")
plt.ylabel("Average INR to USD Exchange Rate")
plt.title("INR Weakening Trend Over Years (MapReduce Analysis)")
plt.grid()
plt.show()
